## 2. Descriptive Analysis
For descriptive analysis, predominantly Pandas is used for data transformation, in conjunction with some useful visualization libraries, which help understanding of the obtained data.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import re
import seaborn

# Set some specs for plotting
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')

# Read data from the data folder
race_df = pd.read_csv('data/race.csv', low_memory=False, index_col=0)
horse_df = pd.read_csv('data/horse.csv', low_memory=False, index_col=0)
individual_df = pd.read_csv('data/individual.csv', low_memory=False, index_col=0)
trainer_df = pd.read_csv('data/trainer.csv', low_memory=False, index_col=0)
jockey_df = pd.read_csv('data/jockey.csv', low_memory=False, index_col=0)

## 2.1 Race Related Analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn

seaborn.barplot(x=race_df['distance'].apply(lambda x: re.search(r'(\d+)m', x).group(0)).value_counts()[:10].index,
                y=race_df['distance'].apply(lambda x: re.search(r'(\d+)m', x).group(0)).value_counts()[:10],
                color='#103942')